In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

In [2]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [4]:
#validation
import pandas as pd
base_dir  = '/media/tohn/SSD/ImageForTrainTest/validation/'
dataframe = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/validation.csv')
validation_dir = os.path.join(base_dir, 'validation')

# load model

In [5]:
model_dir = '/media/tohn/SSD/trainEffbyB/R1/models/B5_R1_noisy.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

In [6]:
#Train
train_df = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/train.csv')
base_dir0 = '/media/tohn/SSD/ImageForTrainTest/'
os.chdir(base_dir)
train_dir = os.path.join(base_dir0, 'train')

# #validation
# valframe = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/validation.csv')
# validation_dir = os.path.join(base_dir, 'validation')

# Prediction

In [7]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

Found 405 validated image filenames belonging to 2 classes.


In [8]:
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

{0: 'Abnormal', 1: 'Normal'}


In [9]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model.predict([x])
    val = -1
    idx = -1
    
    return np.argmax(result[0])

pred_list = list()
for idx,f in enumerate(list(dataframe['filename'])):
    pred_list.append(labels[predict_image(os.path.join(base_dir,f))])
    print(f'{idx}/{len(dataframe)} -- {f}')

dataframe['category'] = pred_list

0/405 -- AB01 P2 C003.JPG
1/405 -- AB01 P3-1 C007.JPG
2/405 -- AB01 P2 C005.JPG
3/405 -- AB01 P5-1 C009.JPG
4/405 -- AB01 P3-1 C003.JPG
5/405 -- AB01 P4-2 C008.JPG
6/405 -- AB01 P3-1 C009.JPG
7/405 -- AB01 P2 C002.JPG
8/405 -- AB01 P4-2 C007.JPG
9/405 -- AB01 P5-2 C009.JPG
10/405 -- AB01 P3-1 C001.JPG
11/405 -- AB01 P4-1 C002.JPG
12/405 -- AB01 P1 C003.JPG
13/405 -- AB01 P2 C004.JPG
14/405 -- AB01 P2 C009.JPG
15/405 -- AB01 P5-2 C004.JPG
16/405 -- AB01 P2 C010.JPG
17/405 -- AB01 P5-2 C010.JPG
18/405 -- AB01 P4-1 C005.JPG
19/405 -- AB01 P4-1 C006.JPG
20/405 -- AB01 P1 C001.JPG
21/405 -- AB01 P3-1 C005.JPG
22/405 -- AB01 P4-2 C002.JPG
23/405 -- AB01 P2 C006.JPG
24/405 -- AB01 P4-2 C010.JPG
25/405 -- AB01 P1 C002.JPG
26/405 -- AB01 P1 C004.JPG
27/405 -- AB01 P4-2 C001.JPG
28/405 -- AB01 P3-1 C006.JPG
29/405 -- AB01 P2 C007.JPG
30/405 -- AB01 P5-1 C007.JPG
31/405 -- AB01 P5-1 C002.JPG
32/405 -- AB01 P5-1 C010.JPG
33/405 -- AB01 P1 C007.JPG
34/405 -- AB01 P5-2 C002.JPG
35/405 -- AB01 P1 C01

282/405 -- AB11 P7-2 C007.JPG
283/405 -- AB11 P7-2 C002.JPG
284/405 -- AB11 P7-2 C004.JPG
285/405 -- AB11 P5-2 C010.JPG
286/405 -- AB11 P5-2 C009.JPG
287/405 -- AB11 P5-2 C006.JPG
288/405 -- AB11 P5-2 C008.JPG
289/405 -- AB11 P7-2 C001.JPG
290/405 -- P1_Case_005.jpg
291/405 -- P1_Case_009.jpg
292/405 -- P1_Case_006.jpg
293/405 -- P1_Case_004.jpg
294/405 -- P1_Case_003.jpg
295/405 -- P1_Case_002.jpg
296/405 -- P1_Case_008.jpg
297/405 -- P1_Case_010.jpg
298/405 -- P1_Case_007.jpg
299/405 -- P1_Case_001.jpg
300/405 -- P2_Case_008.jpg
301/405 -- P2_Case_006.jpg
302/405 -- P2_Case_010.jpg
303/405 -- P2_Case_002.jpg
304/405 -- P2_Case_009.jpg
305/405 -- P2_Case_004.jpg
306/405 -- P2_Case_007.jpg
307/405 -- P2_Case_001.jpg
308/405 -- P2_Case_005.jpg
309/405 -- P2_Case_003.jpg
310/405 -- P3_Case_006-1.jpg
311/405 -- P3_Case_009-1.jpg
312/405 -- P3_Case_004-2.jpg
313/405 -- P3_Case_007-1.jpg
314/405 -- P3_Case_007-2.jpg
315/405 -- P3_Case_002-1.jpg
316/405 -- P3_Case_008-1.jpg
317/405 -- P3_Cas

In [10]:
# dataframe.to_csv('result3_p4.csv',index=False)

In [11]:
# result = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/validation/result3_p4.csv')
# result

# Visualization


In [12]:
data_train = dataframe
#เช็คคลาสใน Predicted
pred_class = set(data_train['category'])
print('Predicted : ',len(pred_class))
print(pred_class)
#เช็คคลาสใน Actual
classe = set(data_train['Class'])
print('Actual : ',len(classe))
print(classe)

Predicted :  2
{'Abnormal', 'Normal'}
Actual :  2
{'Abnormal', 'Normal'}


In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix
act = data_train['Class'].array
pred = data_train['category'].array

In [14]:
cmat = confusion_matrix(act, pred)
# print('fan classifier accuracy = {}%'.format((100.*numpy.trace(cmat))/(numpy.sum(cmat))))
print('fan classifier accuracy = {}%'.format((100.*np.trace(cmat))/(np.sum(cmat))))

fan classifier accuracy = 88.14814814814815%


In [15]:
#Marking the Confusion Matrix
from sklearn.metrics import classification_report,confusion_matrix
# confusion_matrix(act, pred)
print(classification_report(act, pred))#performance

              precision    recall  f1-score   support

    Abnormal       0.98      0.85      0.91       290
      Normal       0.72      0.96      0.82       115

    accuracy                           0.88       405
   macro avg       0.85      0.90      0.87       405
weighted avg       0.91      0.88      0.89       405

